In [83]:
import os
import json
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

# Latest LangChain-style imports
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Token usage callback (location depends on LangChain version)
try:
    from langchain_community.callbacks import get_openai_callback
except Exception:
    from langchain.callbacks import get_openai_callback


In [84]:
load_dotenv()

True

In [85]:
key = os.getenv("OPENAI_API_KEY")

In [86]:
assert key, "Missing OPENAI_API_KEY in your environment (.env)"

In [87]:
# API key is read from environment; do not pass/print it
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

In [88]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [89]:
QUIZ_TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [90]:
quiz_prompt = ChatPromptTemplate.from_template(QUIZ_TEMPLATE)
parser = StrOutputParser()

# generation runnable: inputs -> string
quiz_chain = quiz_prompt | llm | parser

In [91]:
VERIFY_TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

verify_prompt = ChatPromptTemplate.from_template(VERIFY_TEMPLATE)

# Ensure these exist even if earlier cells weren't run
parser = globals().get("parser") or StrOutputParser()
RunnableLambda = globals().get("RunnableLambda")
RunnablePassthrough = globals().get("RunnablePassthrough")
if RunnableLambda is None or RunnablePassthrough is None:
    from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Ensure `quiz_chain` exists even if the quiz cell wasn't run
quiz_chain = globals().get("quiz_chain")
if quiz_chain is None:
    QUIZ_TEMPLATE_LOCAL = globals().get("QUIZ_TEMPLATE")
    if not QUIZ_TEMPLATE_LOCAL:
        raise NameError("QUIZ_TEMPLATE is not defined. Run the quiz template cell first.")
    quiz_prompt = ChatPromptTemplate.from_template(QUIZ_TEMPLATE_LOCAL)
    quiz_chain = quiz_prompt | llm | parser

review_chain = verify_prompt | llm | parser

# Full pipeline: generate quiz, then review using the quiz output
review_from_quiz = RunnableLambda(lambda x: review_chain.invoke({"subject": x["subject"], "quiz": x["quiz"]}))

generate_evaluate_chain = (
    RunnablePassthrough.assign(quiz=quiz_chain)
    .assign(review=review_from_quiz)
)

In [92]:
repo_root = Path.cwd().parent if Path.cwd().name == "experiment" else Path.cwd()
file_path = repo_root / "data.txt"

In [93]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [94]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [95]:
NUMBER=5 
SUBJECT="history"
TONE="sophisticated"


In [96]:

# Token usage tracking
with get_openai_callback() as cb:
    response = generate_evaluate_chain.invoke(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON),
        }
    )

response


{'text': 'Napoleon Bonaparte was a French military leader who rose to prominence during the French Revolution and became one of the most influential figures in European history. Born in 1769 on the island of Corsica, he quickly advanced through the army due to his strategic skill and leadership. In 1804, Napoleon crowned himself Emperor of France, marking the beginning of a period of expansion in which he conquered much of Europe. His rule introduced important reforms, including the Napoleonic Code, which standardized laws and influenced legal systems worldwide. Despite his early successes, Napoleon’s ambitions eventually led to defeat, and after losing key battles such as Waterloo in 1815, he was exiled, bringing an end to his reign.',
 'number': 5,
 'subject': 'history',
 'tone': 'sophisticated',
 'response_json': '{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {

In [97]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")



Total Tokens:1578
Prompt Tokens:892
Completion Tokens:686
Total Cost:0.0005453999999999999


In [98]:

quiz_str = response.get("quiz")
quiz_dict = json.loads(quiz_str) if isinstance(quiz_str, str) else quiz_str

In [99]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value.get("mcq", "")
    options_dict = value.get("options", {}) or {}
    options = " | ".join([f"{option}: {option_value}" for option, option_value in options_dict.items()])
    correct = value.get("correct", "")
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

df = pd.DataFrame(quiz_table_data)

out_dir = repo_root / "experiment"
out_dir.mkdir(parents=True, exist_ok=True)
out_csv = out_dir / "mcq_pipeline_results.csv"
df.to_csv(out_csv, index=False)

print(f"Saved CSV -> {out_csv}")
df.head()

Saved CSV -> /Users/sahbikardi/Desktop/pythonProjects/MCQGENERATOR/experiment/mcq_pipeline_results.csv


,MCQ,Choices,Correct
0,In which year did Napoleon Bonaparte crown him...,a: 1799 | b: 1804 | c: 1812 | d: 1815,b
1,What significant legal reform is attributed to...,a: The Magna Carta | b: The Napoleonic Code | ...,b
2,Which battle marked a pivotal defeat for Napol...,a: Battle of Austerlitz | b: Battle of Leipzig...,c
3,What island was Napoleon Bonaparte born on?,a: Sicily | b: Corsica | c: Crete | d: Malta,b
4,What was one of the primary reasons for Napole...,a: His lack of military experience | b: His fa...,c


In [100]:
import traceback
try:
    c=1/0
except Exception as e:
        traceback.print_exception(type(e), e, e.__traceback__)



Traceback (most recent call last):
  File "/var/folders/hg/h_60tcnj7t976mkq9bqbp_zc0000gn/T/ipykernel_31614/1601091277.py", line 3, in <module>
    c=1/0
ZeroDivisionError: division by zero


In [101]:
# Safely inspect a traceback if available
# (Cell 17 didn't save `tb`, so we grab it from the last exception if possible.)

tb = globals().get("tb")

if tb is None:
    try:
        tb = __import__("sys").exc_info()[2]
    except Exception:
        tb = None

if tb is None:
    print("No traceback object available. (Run an exception cell that assigns `tb = e.__traceback__`.)")
else:
    while tb is not None:
        frame = tb.tb_frame
        filename = frame.f_code.co_filename
        lineno = frame.f_lineno
        function_name = frame.f_code.co_name

        print(f"File '{filename}', line {lineno}, in {function_name}")
        tb = tb.tb_next  # Move to the next frame in the traceback


No traceback object available. (Run an exception cell that assigns `tb = e.__traceback__`.)


In [ ]:
import logging
import os
from datetime import datetime



LOG_FORMAT = f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.log"
print(LOG_FORMAT)